In [28]:
import requests
from bs4 import BeautifulSoup as bs 

# get the web page (https://books.toscrape.com/) html code
get_html = requests.get('https://books.toscrape.com/catalogue/page-1.html')
if get_html.status_code == 200:
    soup = bs(get_html.content, 'html.parser') 
    # find a link to the next page of the site
    next_page = 'https://books.toscrape.com/catalogue/' + soup.find('li', attrs={'class': 'next'}).find('a')['href']
    
    get_next_html = requests.get(next_page)   # get next simple html page
    #if get_next_html.status_code == 200:
        #print(get_next_html.content)

In [29]:
# get get image, title and price from the page
def get_books(content):
    soup = bs(content, 'html.parser')             # make a soup from the html page
    ol = soup.find('ol', attrs={'class': 'row'})  # find a list of books on the page
    books = ol.select('li')                       # find all book on the page
    # make a dict; get image, title and price; add them to the dict for each book
    books_data = []
    for book in books:
        image = 'https://books.toscrape.com/' + book.find('div', attrs={'class': 'image_container'}).find('img')['src']
        title = book.find('h3').find('a')['title']
        price = book.find('p', attrs={'class': 'price_color'}).text
        
        book_dict = {
            'image': image,
            'title': title,
            'price': price
        }
        books_data.append(book_dict)
    return books_data

In [30]:
# find the next page of the site https://books.toscrape.com/
def get_next_page(content):
    soup = bs(content, 'html.parser')
    try:
        next_page = 'https://books.toscrape.com/catalogue/' + soup.find('li', attrs={'class': 'next'}).find('a')['href']
        return next_page
    except:
        pass

In [ ]:
# main script
final_data = []
page_number = 1   # start scrapping always from the 1st page
url = 'https://books.toscrape.com/catalogue/page-1.html'
get_html = requests.get(url)

if get_html.status_code == 200:
    
    while True:
        books = get_books(get_html.content)
        print(f'We have got {len(books)} books from the page number {page_number}')
        final_data += books

        next_page = get_next_page(get_html.content)
        if next_page: 
            page_number += 1
            get_html = requests.get(next_page)
            if get_html.status_code == 200:
                print(f'Starts scraping {page_number}')
        else:
            break

print(f'Scraping completed! Total: {page_number} pages with {len(final_data)} books.')
